## Exercise 1: Exceptional Olympians

Scrape data from [this wikipedia site](https://en.wikipedia.org/wiki/List_of_multiple_Olympic_medalists) about exceptional Olympic medalists. 

1. Download the html using urllib. 
2. Parse this html with BeautifulSoup.
3. Extract the html that corresponds to the big table from the soup.
4. Parse the table into a pandas dataframe. Hint: both the "No." and the "Total." column use row-spans which are tricky to parse, both with a pandas reader and manually. For the purpose of this exercise, exclude all rows that are not easy to parse (the first one is Bjørn Dæhlie).
5. Create a table that shows for each country how many gold, silver, bronze, and total medals it won in that list.

In [4]:
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd

Download the html using urllib. 
Parse this html with BeautifulSoup.

In [5]:
url = "https://en.wikipedia.org/wiki/List_of_multiple_Olympic_medalists"

req = urllib.request.Request(url)
with urllib.request.urlopen(req) as response:
    html = response.read()

print(html)

class_soup = BeautifulSoup(html, 'html.parser')

b'<!DOCTYPE html>\n<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-zebra-design-disabled vector-feature-custom-font-size-clientpref-0 vector-feature-client-preferences-disabled vector-feature-typography-survey-disabled vector-toc-available" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8">\n<title>List of multiple Olympic medalists - Wikipedia</title>\n<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-f

Extract the html that corresponds to the big table from the soup.

In [6]:
# we can retreive all tables, our desired table is the first one:
table_html = class_soup("table")[0]
table_html

<table class="wikitable sortable">
<tbody><tr>
<th>No.
</th>
<th style="width:7.8em;">Athlete
</th>
<th style="width:8.2em;">Nation
</th>
<th style="width:5.6em;">Sport
</th>
<th>Years
</th>
<th>Games
</th>
<th>Gender
</th>
<th style="background-color:gold; width:3.5em; font-weight:bold;">Gold
</th>
<th style="background-color:silver; width:3.5em; font-weight:bold;">Silver
</th>
<th style="background-color:#cc9966; width:3.5em; font-weight:bold;">Bronze
</th>
<th style="width:3.5em;">Total
</th></tr>
<tr>
<td>1
</td>
<td align="left"><span data-sort-value="Phelps, Michael"><span class="vcard"><span class="fn"><a href="/wiki/Michael_Phelps" title="Michael Phelps">Michael Phelps</a></span></span></span></td>
<td align="left"><span class="mw-image-border" typeof="mw:File"><span><img alt="" class="mw-file-element" data-file-height="650" data-file-width="1235" decoding="async" height="12" src="//upload.wikimedia.org/wikipedia/en/thumb/a/a4/Flag_of_the_United_States.svg/22px-Flag_of_the_Unit

Parse the table into a pandas dataframe. 

In [20]:
athletes = pd.read_html(str(table_html))[0]
#athletes = pd.read_html(str(html))[3]
#athletes = pd.read_html(str(table_html), header=0)[0]
athletes.head(15)

,No.,Athlete,Nation,Sport,Years,Games,Gender,Gold,Silver,Bronze,Total
0,1,Michael Phelps,United States,Swimming,2004–2016,Summer,M,23,3,2,28
1,2,Larisa Latynina,Soviet Union,Gymnastics,1956–1964,Summer,F,9,5,4,18
2,3,Marit Bjørgen,Norway,Cross-country skiing,2002–2018,Winter,F,8,4,3,15
3,4,Nikolai Andrianov,Soviet Union,Gymnastics,1972–1980,Summer,M,7,5,3,15
4,5,Ole Einar Bjørndalen,Norway,Biathlon,1998–2014,Winter,M,8,4,1,13
5,6,Boris Shakhlin,Soviet Union,Gymnastics,1956–1964,Summer,M,7,4,2,13
6,7,Edoardo Mangiarotti,Italy,Fencing,1936–1960,Summer,M,6,5,2,13
7,7,Ireen Wüst,Netherlands,Speed skating,2006–2022,Winter,F,6,5,2,13
8,9,Takashi Ono,Japan,Gymnastics,1952–1964,Summer,M,5,4,4,13
9,10,Paavo Nurmi,Finland,Athletics,1920–1928,Summer,M,9,3,0,12


For cases where the row is screwed up, the bronze column is NaN, which we can use to filter:

In [8]:
athletes = athletes[pd.notnull(athletes["Bronze"])]

Subset to the relevant columns: 

In [9]:
athletes = athletes[["Nation", "Gold", "Silver", "Bronze"]]
athletes.head(10)

,Nation,Gold,Silver,Bronze
0,United States,23,3,2
1,Soviet Union,9,5,4
2,Norway,8,4,3
3,Soviet Union,7,5,3
4,Norway,8,4,1
5,Soviet Union,7,4,2
6,Italy,6,5,2
7,Netherlands,6,5,2
8,Japan,5,4,4
9,Finland,9,3,0


Grouping, summing, calculating the total, and sorting: 

In [10]:
countries = athletes.groupby("Nation").sum()

In [11]:
countries["Total"] = countries["Gold"] + countries["Silver"] + countries["Bronze"]
countries.sort_values("Total", ascending=False)

,Gold,Silver,Bronze,Total
Nation,,,,
United States,162,62,46,270
Soviet Union,55,50,29,134
Norway,59,31,18,108
Germany,38,34,20,92
Australia,39,31,19,89
Japan,42,27,19,88
Italy,27,25,22,74
Hungary,38,20,14,72
Finland,30,13,14,57


## Exercise 2 – APIs

Use the [Open Notify API](http://open-notify.org/Open-Notify-API/People-In-Space/) to find out how many people are in space right now.

In [1]:
import requests 

url = "http://api.open-notify.org/astros.json"

r = requests.get(url)
data = r.json()
data

{'people': [{'craft': 'ISS', 'name': 'Andrew Morgan'},
  {'craft': 'ISS', 'name': 'Oleg Skripochka'},
  {'craft': 'ISS', 'name': 'Jessica Meir'}],
 'message': 'success',
 'number': 3}